# Fast Async Code with Cython and AsyncIO

_by Stefan Behnel & Anton Caceres_

Stefan  (<stb@skoobe.de>)
-  Python developer since 2002
-  Cython core developer, main developer of lxml

Anton  (<ac@skoobe.de>)
-  Python developer since 2010
-  Organiser of PyCon-DE 2016 (this October!)

Skoobe
-------

* eBook subscription service since 2012
* leading offer in Germany, available worldwide
* ~200 000 books for 9.99€ / month
* Android, iOS apps - 5 stars
* Python based backend
* young and fascinating place to work

![Alt text](images/skoobe.jpg)

## What to expect from this talk

* Introduction to AsyncIO
* Introduction to Cython
* Async coroutines with Cython
* Practical examples

## Introduction to AsyncIO

AsyncIO
-------

* Module for writing async code with high concurrency
* Infrastructure for writing single-threaded concurrent code using coroutines
* Runs on Python 3.3+, backport to Py2 available
* Details in another notebook...

### Delays due to processing time

<img src="images/delayed_async_requests-1.png" width="370px">

### Can be even worse

<img src="images/delayed_async_requests-2.png" width="370px">

### Cutting processing time

<img src="images/delayed_async_requests-3.png" width="370px">

## Faster processing reduces latency in I/O applications

* IOLoop runs in one thread
* Only one task runs at any given moment
* No I/O while processing
* The faster we process each task, the quicker we respond to others

Cython
------

* Most widely used static Python compiler
* Major part of Scientific Python ecosystem
* Outputs C code using the CPython C-API
* Extends Python language for optimisation and C/C++ interaction
* Open-source: cython.org / github

### Features:
* freely mix Python and C/C++
* benefit from static code optimisation
* wide range of manual tuning capabilities
* highly portable, supports CPython 2.6 - 3.6+

-  Cython syntax generally follows Python 2.7

   -  plus all from Py3 that doesn't conflict

-  Switch to full Python 3 syntax/semantics by directive:

   `# cython: language_level=3`

-  Supports PEP 492 coroutines with async/await

   -  slower in Py2.6/7 than in Py3

## Cython in 10 Minutes

# Async Cython Coroutines with Python 3.5

In [ ]:
%load_ext cython

In [ ]:
import asyncio

In [ ]:
def run_async(coro):
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(coro)
    return result

## Cython coroutines are fully compatible with asyncio and Python coroutines

In [ ]:
%%cython
# cython: language_level = 3

async def add_one(fut):
    result = await fut
    print("Cy-ADD:", result)
    return result + 1

async def one():
    return 1

In [ ]:
run_async(add_one(add_one(one())))

In [ ]:
async def py_add_one(fut):
    result = await fut
    print("Py-ADD:", result)
    return result + 1

In [ ]:
run_async(add_one(py_add_one(one())))

## Let's play some ping-pong
-  one async Python function prints "ping"
-  one async Cython function prints "pong"
-  jump between the two based on a counter

In [ ]:
%%cython

# example coro_map: {0: cy_dec1, 1: cy_dec1}

async def cy_dec1(coro_map, value, show=False):
    if show:
        print('pong')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
# example coro_map: {0: py_dec1, 1: py_dec1}

async def py_dec1(coro_map, value, show=False):
    if show:
        print('ping')
    if value > 0:
        value = await coro_map[value % 2](coro_map, value - 1, show)
    return value

In [ ]:
run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 6, show=True))

## How fast are Cython coroutines?

In [ ]:
%timeit run_async(py_dec1({0: py_dec1, 1: py_dec1}, 600)) # python-only

In [ ]:
%timeit run_async(cy_dec1({0: cy_dec1, 1: cy_dec1}, 600)) # cython-only

In [ ]:
%timeit run_async(py_dec1({0: cy_dec1, 1: py_dec1}, 600)) # alternate

## Fizz-Buzz example

# Practical Example: Decryption Server

### Simple setup where client sends encrypted data stream to the server

![Alt text](http://i.imgur.com/8NB5zVe.jpg)

## _Decryption Server: code_
_(another notebook)_

## Thank you!

### Questions?

_Stefan: stb@skoobe.de, Anton: ac@skoobe.de_